In [1]:
from pyspark.sql import SparkSession

MAX_MEMORY = "5g" # config("spark.executor.memory", MAX_memory), config("spark.driver.meemory", MAX_MEMORY)

spark = SparkSession.builder.master('local[8]').appName('movie-recommendation').getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/12 18:10:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
rating_file = '/Users/lifelog/_PML/fc_spark/data/ml-25m/ratings.csv'
rating_df = spark.read.csv(f"file:///{rating_file}", inferSchema=True, header=True)
rating_df.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



In [3]:
rating_df = rating_df.select(['userId', 'movieId', 'rating'])

In [4]:
rating_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [5]:
rating_df.select(['rating']).describe().show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|          25000095|
|   mean| 3.533854451353085|
| stddev|1.0607439611423508|
|    min|               0.5|
|    max|               5.0|
+-------+------------------+



In [6]:
train_df, test_df = rating_df.randomSplit([0.8, 0.2])

In [7]:
from pyspark.ml.recommendation import ALS

als = ALS(maxIter=10,
          regParam = 0.1,
          userCol='userId',
          itemCol='movieId',
          ratingCol='rating',
          coldStartStrategy='drop' # nan or drop
          )

In [8]:
als_model = als.fit(train_df)

23/11/12 18:11:41 WARN BlockManager: Block rdd_39_1 could not be removed as it was not found on disk or in memory
23/11/12 18:11:41 WARN BlockManager: Block rdd_39_5 could not be removed as it was not found on disk or in memory
23/11/12 18:11:41 WARN BlockManager: Block rdd_40_5 could not be removed as it was not found on disk or in memory
23/11/12 18:11:41 ERROR Executor: Exception in task 5.0 in stage 9.0 (TID 41)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3793)
	at scala.collection.mutable.ArrayBuilder$ofInt.mkArray(ArrayBuilder.scala:339)
	at scala.collection.mutable.ArrayBuilder$ofInt.result(ArrayBuilder.scala:383)
	at scala.collection.mutable.ArrayBuilder$ofInt.result(ArrayBuilder.scala:330)
	at org.apache.spark.ml.recommendation.ALS$UncompressedInBlockBuilder.build(ALS.scala:1452)
	at org.apache.spark.ml.recommendation.ALS$.$anonfun$makeBlocks$5(ALS.scala:1660)
	at org.apache.spark.ml.recommendation.ALS$$$Lambda$4043/0x000000080

ConnectionRefusedError: [Errno 61] Connection refused

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/lifelog/miniconda3/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/lifelog/miniconda3/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/lifelog/miniconda3/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [15]:
predictions = als_model.transform(test_df)

In [16]:
predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     1|    306|   3.5| 4.0662317|
|     1|   1260|   3.5| 4.0207963|
|     1|   2068|   2.5| 3.8897588|
|     1|   3949|   5.0|  3.921198|
|     1|   4325|   5.0|   3.98439|
|     1|   5952|   4.0| 3.6899786|
|     1|   6711|   5.0|  3.989763|
|     1|   7327|   3.5| 4.0020003|
|     1|   7361|   5.0|  4.146472|
|     1|   7937|   3.0| 3.8399374|
|     1|   8729|   3.5| 3.0515068|
|     1|   8786|   4.0| 3.7612777|
|     1|   8973|   4.0| 3.8708837|
|     1|  27266|   4.5| 3.7075858|
|    12|     16|   5.0| 3.5409687|
|    12|     20|   1.0|  2.143339|
|    12|    101|   4.0|  3.627047|
|    12|    185|   2.0| 2.3618746|
|    12|    186|   2.5| 2.2741473|
|    12|    222|   3.0| 3.2050283|
+------+-------+------+----------+
only showing top 20 rows



In [17]:
predictions.select(['rating', 'prediction']).describe().show()

+-------+------------------+------------------+
|summary|            rating|        prediction|
+-------+------------------+------------------+
|  count|           4997655|           4997655|
|   mean|3.5341622220821565|3.4138912451126617|
| stddev|1.0609444014040805|0.6459231419320535|
|    min|               0.5|          -1.25035|
|    max|               5.0|         6.6312466|
+-------+------------------+------------------+



In [18]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol='prediction',
                    labelCol='rating',
                    metricName='rmse',)

In [19]:
rmse = evaluator.evaluate(predictions)
print(rmse)

0.8069730161822684


In [20]:
# als_model.recommendForAllUsers(3).show()
als_model.recommendForUserSubset()

ERROR:root:KeyboardInterrupt while sending command.               (2 + 4) / 100]
Traceback (most recent call last):
  File "/Users/lifelog/miniconda3/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/lifelog/miniconda3/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/lifelog/miniconda3/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
als_model.recommendForAllItems(3).show()

In [24]:
from pyspark.sql.types import IntegerType

user_list = [65, 78, 81]
spark.createDataFrame(user_list, IntegerType()).toDF('userId').toDF('userId'))